In [1]:
import numpy as np
import sys
import os

In [2]:
import plotly.graph_objects as go

In [3]:
sys.path.append(os.path.abspath(os.path.join(".", "..")))
import bssneqs as bssn
import tomllib
from nwave import *

In [4]:
with open("params.toml", "rb") as f:
    params = tomllib.load(f)

In [5]:
# Set up grid
if "Xmin" in params and params["Xmin"] < 0.0:
    extended_domain = True
    cellgrid = False
    if params["Nx"] % 2 != 0:
        params["Nx"] += 1
        print(f"Adjusted Nx to {params['Nx']} (must be even)")
else:
    extended_domain = False
    cellgrid = True

nghost = params["NGhost"]
g = Grid1D(params, cell_centered=cellgrid)

In [6]:
def get_filter_type(filter_str):
    try:
        return filter_type_map[filter_str]
    except KeyError:
        raise ValueError(f"Unknown filter string: '{filter_str}'")

In [7]:
r = g.xi[0]
dr = g.dx[0]
if params["D1"] == "E4":
    D1 = ExplicitFirst44_1D(dr)
    g.set_D1(D1)
elif params["D1"] == "E6":
    D1 = ExplicitFirst642_1D(dr)
    g.set_D1(D1)
elif params["D1"] == "JP6":
    #D1 = CompactFirst1D(r, "D1_JTP6", method="LUSOLVE")
    D1 = CompactFirst1D(r, DerivType.D1_JP6, CFDSolve.LUSOLVE)
    g.set_D1(D1)
elif params["D1"] == "KP4":
    # D1 = CompactFirst1D(r, "D1_KP4", method="LUSOLVE")
    D1 = CompactFirst1D(r, DerivType.D1_KP4, CFDSolve.LUSOLVE)
    g.set_D1(D1)
else:
    raise NotImplementedError("D1 = { E4, E6, JP6, KP4 }")

In [8]:
if params["D2"] == "E4":
    D2 = ExplicitSecond44_1D(dr)
    g.set_D2(D2)
elif params["D2"] == "E6":
    D2 = ExplicitSecond642_1D(dr)
    g.set_D2(D2)
elif params["D2"] == "JP6":
    D2 = CompactSecond1D(r, DerivType.D2_JP6, CFDSolve.LUSOLVE)
    #D2 = CompactSecond1D(r, DerivType.D2_JP6, CFDSolve.D_LU)
    g.set_D2(D2)
else:
    raise NotImplementedError("D2 = { E4, E6, JP6 }")

In [9]:
filter_str = params["Filter"]
ftype = get_filter_type(filter_str)
frequency = params.get("FilterFrequency",1)
alpha = params.get("FilterAlpha", 0.4)
beta = params.get("FilterBeta", 0.0)
fbounds = params.get("FilterBoundary", False)
sigma = params.get("FilterKOsigma", 0.1)

if ftype == "KO6":
    bssn_filter = KreissOligerFilterO6_1D(dr, sigma, filter_boundary=fbounds)
    g.set_filter(bssn_filter)
elif ftype == "KO8":
    print("Setting KO8 filter")
    bssn_filter = KreissOligerFilterO8_1D(dr, sigma, filter_boundary=fbounds)
    g.set_filter(bssn_filter)
elif ftype in CompactFilterTypes:
    print("Setting a compact filter")
    apply_filter = FilterApply.APPLY_VARS
    method = CFDSolve.SCIPY
    bssn_filter = NCompactFilter.init_filter(r, ftype, apply_filter, method, frequency, fbounds, alpha, beta)
    g.set_filter(bssn_filter)
    print(f"bssn_filter type = {bssn_filter.get_filter_type()}")
elif ftype == "None":
    print("No filter found. Setting to 'None'")
    bssn_filter = None
    g.set_filter(bssn_filter)
else:
    raise NotImplementedError("Filter = { KO6, KO8, JTT6, JTP6, JTT8, JTP8, KP4 }")

Setting a compact filter
bssn_filter type = FilterType.JTT8


In [10]:
print(f"D1 type: {g.D1.get_type()}, method: {g.D1.get_method()}")
print(f"D2 type: {g.D2.get_type()}, method: {g.D2.get_method()}")
print(f"filter_string = {filter_str}")
print(f"ftype = {ftype}")

print(f"Filter type: {g.Filter[0].get_filter_type()}")
print(f"Filter apply: {g.Filter[0].get_apply_filter()}")

D1 type: DerivType.D1_JP6, method: CFDSolve.LUSOLVE
D2 type: DerivType.D2_JP6, method: CFDSolve.LUSOLVE
filter_string = JTT8
ftype = FilterType.JTT8
Filter type: FilterType.JTT8
Filter apply: FilterApply.APPLY_VARS


In [11]:
# GBSSN system: (sys, lapse advection, shift advection)
#    sys = 0 (Eulerian), 1 (Lagrangian)
sys = bssn.GBSSNSystem(1, 1, 1)
eqs = bssn.BSSN(
    g, params["Mass"], params["eta"], extended_domain, "FUNCTION", sys, have_d2=True
)
eqs.initialize(g, params)

In [12]:
r = g.xi[0]

In [13]:
eqs.cal_constraints(eqs.u, g)

/Users/dave/projects/pynwave2d/nwave/compactderivs.py:578: RuntimeWarning: divide by zero encountered in matmul
  rhs = np.matmul(self.B, f)
/Users/dave/projects/pynwave2d/nwave/compactderivs.py:578: RuntimeWarning: overflow encountered in matmul
  rhs = np.matmul(self.B, f)
/Users/dave/projects/pynwave2d/nwave/compactderivs.py:578: RuntimeWarning: invalid value encountered in matmul
  rhs = np.matmul(self.B, f)


In [14]:
d_chi = D1.grad(eqs.u[eqs.U_CHI])
d_g_rr = D1.grad(eqs.u[eqs.U_GRR])
d_g_tt = D1.grad(eqs.u[eqs.U_GTT])
d_A_rr = D1.grad(eqs.u[eqs.U_ARR])
d_K = D1.grad(eqs.u[eqs.U_K])
d2_chi = D2.grad2(eqs.u[eqs.U_CHI])
d2_g_tt = D2.grad2(eqs.u[eqs.U_GTT])
d_alpha = D1.grad(eqs.u[eqs.U_ALPHA])
d2_alpha = D2.grad2(eqs.u[eqs.U_ALPHA])
Gt = eqs.u[eqs.U_GT]
d_Gt = D1.grad(Gt)

/Users/dave/projects/pynwave2d/nwave/compactderivs.py:578: RuntimeWarning: divide by zero encountered in matmul
  rhs = np.matmul(self.B, f)
/Users/dave/projects/pynwave2d/nwave/compactderivs.py:578: RuntimeWarning: overflow encountered in matmul
  rhs = np.matmul(self.B, f)
/Users/dave/projects/pynwave2d/nwave/compactderivs.py:578: RuntimeWarning: invalid value encountered in matmul
  rhs = np.matmul(self.B, f)


In [15]:
# our custom event handler
def update_trace(trace, points, selector):
    # this list stores the points which were clicked on
    # in all but one trace they are empty
    if len(points.point_inds) == 0:
        return          
    for i,_ in enumerate(fig.data):
        fig.data[i]['line']['width'] = default_linewidth + highlighted_linewidth_delta * (i == points.trace_index)

default_linewidth = 2
highlighted_linewidth_delta = 2

fig = go.FigureWidget()
fig.layout.hovermode = 'closest'
fig.layout.hoverdistance = -1 #ensures no "gaps" for selecting sparse data
fig.add_trace(go.Scatter(x=r, y=Gt, name="Gt", mode="lines",line={ 'width': default_linewidth }))
fig.add_trace(go.Scatter(x=r, y=d_Gt, name="d_Gt", mode="lines+markers", line={ 'width': default_linewidth }))
# we need to add the on_click event to each trace separately       
for i in range( len(fig.data) ):
    fig.data[i].on_click(update_trace)

# Layout with log scale option
fig.update_layout(
    title='Gt and dx_Gt',
    xaxis=dict(title='x', type='linear'),  # use 'log' for log scale
    yaxis=dict(title='y'),
    hovermode='x',
    template='plotly_white'
)

fig.show()

FigureWidget({
    'data': [{'line': {'width': 2},
              'mode': 'lines',
              'name': 'Gt',
              'type': 'scatter',
              'uid': 'a56ab65c-b9c7-466b-9970-f7b4c87ea484',
              'x': {'bdata': ('Z2KAPXAmWcCaQQApoBlZwM0ggBTQDF' ... 'AU0AxZQJtBACmgGVlAZ2KAPXAmWUA='),
                    'dtype': 'f8'},
              'y': {'bdata': ('OigESJRblD9zO/ug+GWUPwvAa5tncJ' ... 'ubZ3CUv3I7+6D4ZZS/OigESJRblL8='),
                    'dtype': 'f8'}},
             {'line': {'width': 2},
              'mode': 'lines+markers',
              'name': 'd_Gt',
              'type': 'scatter',
              'uid': '18c8685f-4ad2-4c18-b595-c903f5f1e674',
              'x': {'bdata': ('Z2KAPXAmWcCaQQApoBlZwM0ggBTQDF' ... 'AU0AxZQJtBACmgGVlAZ2KAPXAmWUA='),
                    'dtype': 'f8'},
              'y': {'bdata': ('EpWAZuJPKD8YzW9zaUkqPzXuAHIFAC' ... 'ByBQAqP23cb3NpSSo/82aAZuJPKD8='),
                    'dtype': 'f8'}}],
    'layout': {'hoverdistance': -1,
      

In [16]:
ED1 = ExplicitFirst642_1D(dr)
ED2 = ExplicitSecond642_1D(dr)

In [17]:
ed_chi = ED1.grad(eqs.u[eqs.U_CHI])
ed_g_rr = ED1.grad(eqs.u[eqs.U_GRR])
ed_g_tt = ED1.grad(eqs.u[eqs.U_GTT])
ed_A_rr = ED1.grad(eqs.u[eqs.U_ARR])
ed_K = ED1.grad(eqs.u[eqs.U_K])
ed2_chi = ED2.grad2(eqs.u[eqs.U_CHI])
ed2_g_tt = ED2.grad2(eqs.u[eqs.U_GTT])
ed_Gt = ED1.grad(Gt)

In [18]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=r, y=ed_Gt, name="ed_Gt (642)", mode="lines+markers"))
fig.add_trace(go.Scatter(x=r, y=d_Gt, name="d_Gt", mode="lines+markers"))
#fig.add_trace(go.Scatter(x=r, y=ed_Gt, name="ed_Gt (642)", mode="lines+markers",hoverinfo='skip'))
#fig.add_trace(go.Scatter(x=r, y=d_Gt, name="d_Gt", mode="lines+markers",hoverinfo='skip'))

# we need to add the on_click event to each trace separately       
for i in range( len(fig.data) ):
    fig.data[i].on_click(update_trace)

# Layout with log scale option
fig.update_layout(
    title='Explicit and Compact Derivatives of Gt',
    xaxis=dict(title='x', type='linear'),  # use 'log' for log scale
    yaxis=dict(title='y'),
    hovermode='x',
    template='plotly_white'
)

fig.show()

In [19]:
f = 1/r
amp = 1.0e-1
#fn = f + amp*np.sin(137.523*r)
gn = np.random.normal(0, 0.1, len(r))
fn = f + amp*gn
CF = CompactFilter(r, FilterApply.APPLY_VARS, FilterType.JTT6, 1, alpha=0.4, beta=0.0)
ff = CF.filter(f)

/Users/dave/projects/pynwave2d/nwave/compactfilters.py:122: RuntimeWarning:

divide by zero encountered in matmul

/Users/dave/projects/pynwave2d/nwave/compactfilters.py:122: RuntimeWarning:

overflow encountered in matmul

/Users/dave/projects/pynwave2d/nwave/compactfilters.py:122: RuntimeWarning:

invalid value encountered in matmul



In [20]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=r, y=fn, name="f with noise", mode="lines+markers"))
fig.add_trace(go.Scatter(x=r, y=ff, name="f filtered", mode="lines+markers"))
fig.update_layout(
    title='Plot of F(r) with noise and filtered.  F(r) = 1/r.',
    xaxis=dict(title='x', type='linear'),  # use 'log' for log scale
    yaxis=dict(title='y'),
    xaxis_range=[2,4],
    yaxis_range=[-0.5,0.5],
    hovermode='x',
    template='plotly_white'
)

In [21]:
Gtf = CF.filter(Gt)
d_Gtf = D1.grad(Gtf)

/Users/dave/projects/pynwave2d/nwave/compactfilters.py:122: RuntimeWarning:

divide by zero encountered in matmul

/Users/dave/projects/pynwave2d/nwave/compactfilters.py:122: RuntimeWarning:

overflow encountered in matmul

/Users/dave/projects/pynwave2d/nwave/compactfilters.py:122: RuntimeWarning:

invalid value encountered in matmul

/Users/dave/projects/pynwave2d/nwave/compactderivs.py:578: RuntimeWarning:

divide by zero encountered in matmul

/Users/dave/projects/pynwave2d/nwave/compactderivs.py:578: RuntimeWarning:

overflow encountered in matmul

/Users/dave/projects/pynwave2d/nwave/compactderivs.py:578: RuntimeWarning:

invalid value encountered in matmul



In [22]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=r, y=d_Gt, name="d_Gt", mode="lines+markers"))
fig.add_trace(go.Scatter(x=r, y=d_Gtf, name="d_Gtf", mode="lines+markers"))
fig.update_layout(
    title='Derivative d_Gt and Derivative of Filtered Gtf',
    xaxis=dict(title='x', type='linear'),  # use 'log' for log scale
    yaxis=dict(title='y'),
    xaxis_range=[-1,3],
    hovermode='x',
    template='plotly_white'
)

In [23]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=r, y=Gt, name="Gt", mode="lines+markers"))
fig.add_trace(go.Scatter(x=r, y=Gtf, name="Gtf", mode="lines+markers"))
fig.update_layout(
    title='Gt and Filtered Gtf',
    xaxis=dict(title='x', type='linear'),  # use 'log' for log scale
    yaxis=dict(title='y'),
    xaxis_range=[-1,3],
    hovermode='x',
    template='plotly_white'
)

In [27]:
dd = d_Gt - d_Gtf
fig = go.Figure()
fig.add_trace(go.Scatter(x=r, y=dd, name="Diff d_Gt", mode="lines+markers"))

fig.update_layout(
    title='d_Gt - d_Gtf',
    xaxis=dict(title='x', type='linear'),  # use 'log' for log scale
    yaxis=dict(title='y'),
    xaxis_range=[-1,3],
    hovermode='x',
    template='plotly_white'
)